In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [ ]:
!git clone https://github.com/aykhangaffarov/Generative_Models # this is for loading git with correct brach

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'


In [ ]:
#!mv Generative_Models/data/.CBSD68/CBSD68.tar.bz Generative_Models/data/

# Settings

In [ ]:
import os
print(os.getcwd())

In [ ]:
%env TF_KERAS = 1

sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [ ]:
os.chdir('Generative_Models')
print(os.getcwd())

In [ ]:
!pip -q install -r requirements.txt

# Unzip

In [ ]:
!ls -a data/.CBSD68_low2high_LMDB

In [ ]:
!cat  data/.CBSD68_low2high_LMDB/.CBSD68_low2high_LMDB.tar.*  > data/.CBSD68_low2high_LMDB/.CBSD68_low2high_LMDB.tar

In [ ]:
!tar -xvf data/.CBSD68_low2high_LMDB/.CBSD68_low2high_LMDB.tar

In [ ]:
!ls -a .CBSD68_low2high_LMDB

In [ ]:
#!mv data/.CBSD68_low2high_LMDB/ data/SR

In [ ]:
#!ls data/SR

# **Loading** Dataset

In [ ]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [ ]:
dataset_name='CBSD68_low2high'
lmdb_dir = '.CBSD68_low2high_LMDB'

In [ ]:
batch_size = 10
latents_dim = 50
intermediate_dim = 50

In [ ]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None,
        return_itr=False,
        class_mode='sr'
)

In [ ]:
for data in train_generator:
    break
#data = next(train_itr)
print(data.keys())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

In [ ]:
import numpy as np

In [ ]:
source_img_size, target_img_size =  data['xt0'][0].shape, data['xt1'][0].shape
source_img_size, target_img_size

In [ ]:
#source_img_size, target_img_size = (32, 48, 3), (10, 16, 3)

In [ ]:
Image.fromarray((data['xt0'][0].numpy() * 255.0).astype(np.uint8), mode='RGB')

In [ ]:
image = data['xt1'][0].numpy()

In [ ]:
Image.fromarray((data['xt1'][0].numpy() * 255.0).astype(np.uint8), mode='RGB')

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(source_img_size, Iterable):
    flat_outputs_shape = np.prod(source_img_size)
flat_outputs_shape    

# Model's Layers definition

In [ ]:
kernel_size=3
stride = 2

In [ ]:
c = (55, 75)#list(map(lambda x: x// (stride*stride), source_img_size[:-1]))
c = (*c, intermediate_dim)
c

In [ ]:
np.product(c)

In [ ]:
enc_lays = [
    tf.keras.layers.UpSampling2D(size=(2, 2)),
    tf.keras.layers.UpSampling2D(size=(2, 2)),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product(c), activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=c),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=intermediate_dim, kernel_size=kernel_size, strides=(stride, stride), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=kernel_size, strides=(1, 1), padding="SAME")
]

# Model definition

In [ ]:
model_name = dataset_name+'_Conv_Pre_Upsampling_LMDB'
#windows
#experiments_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'experiments'+sep_local + model_name

#linux 
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [ ]:
target_img_size, source_img_size

In [ ]:
variables_params = \
[
    {
        'name': 'inference',  #'upsampler',
        'inputs_shape':target_img_size,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', #'constructive', 
        'inputs_shape':latents_dim,
        'outputs_shape':source_img_size,
        'layers':dec_lays
    }
]

In [ ]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

In [ ]:
#!mv drive/MyDrive/Colab\ Notebooks/Pre_Upsampling_weights/ experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir
!cp -r weights/Pre_Upsampling_weights/initial17completed/ experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir
!mv experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir/initial17completed/* experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir/

In [ ]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore, #to restore trained model, set filepath=_restore
    episode_len=1
    )

In [ ]:
ae.compile()
#ae.compile(metrics=None)

# Callbacks

In [ ]:
# added for linux warning suppression
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)


from training.callbacks.trace_image_reconstruction import trace_reconstruction
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_reconstuction_dir = os.path.join(experiments_dir, 'image_reconstuction_dir')
create_if_not_exist(image_reconstuction_dir)
absolute = abspath(image_reconstuction_dir)
print("image_reconstuction_dir",absolute)

In [ ]:
img_reconst = trace_reconstruction(filepath=image_reconstuction_dir, image=image, gen_freq=5)

# Model Training

In [ ]:
#ae.compile()

In [ ]:
ae.fit(
    x=train_generator,
    input_kw=None,
    steps_per_epoch=int(1e4),
    epochs=int(1e6), 
    verbose=1,
    callbacks=[ es, ms, csv_log, img_reconst],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
    )

In [ ]:
#!mv drive/MyDrive/Colab\ Notebooks/Pre_Upsampling_weights/ Generative_Models/experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir
!cp -r  experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/var_save_dir Pre_Upsampling_weights/initial35completed/ # number of completed epochs during training session

In [ ]:
#ae.save(_model)

In [ ]:
#ae.compile()

# Testing

In [ ]:
for t_data in val_generator:
    break
#data = next(train_itr)
print(t_data.keys())

In [ ]:
data['xt0'].shape

In [ ]:
t_source_img_size, t_target_img_size =  t_data['xt0'][0].shape, t_data['xt1'][0].shape
t_source_img_size, t_target_img_size

In [ ]:
#source_img_size, target_img_size = (32, 48, 3), (10, 16, 3)

In [ ]:
Image.fromarray((t_data['xt0'][1].numpy() * 255.0).astype(np.uint8), mode='RGB')

In [ ]:
image = t_data['xt1'][1].numpy()

In [ ]:
Image.fromarray((t_data['xt1'][1].numpy() * 255.0).astype(np.uint8), mode='RGB')

In [ ]:
preds=ae.predict(t_data['xt1'])

In [ ]:
#from sklearn.metrics import confusion_matrix

In [ ]:
os.chdir('experiments/CBSD68_low2high_Conv_Pre_Upsampling_LMDB/results')
print(os.getcwd())

In [ ]:
Image.fromarray((preds[1] * 255.0).astype(np.uint8), mode='RGB')

# Saving results

In [ ]:
# if you want to save new results, first remove older images from following directories
#!rm -r combined/*
#!rm -r seperate/*

In [ ]:
# seperate images
os.chdir('seperate')
print(os.getcwd())
i=1
for k in preds:  
  Image.fromarray((k * 255.0).astype(np.uint8), mode='RGB').save('image_result_'+str(i)+'.jpg')
  i+=1
os.chdir('../')
print(os.getcwd())

In [ ]:
#combined with original images
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [ ]:
os.chdir('combined')
print(os.getcwd())
i=0
for k in preds:
  im2=Image.fromarray((k * 255.0).astype(np.uint8), mode='RGB')
  im1=Image.fromarray((t_data['xt0'][i].numpy() * 255.0).astype(np.uint8), mode='RGB')
  get_concat_h(im1, im2).save('_combined_image_result_'+str(i+1)+'_h.jpg')
  i+=1
os.chdir('../')
print(os.getcwd())

# Metrics evaluation

In [ ]:
preds.shape

In [ ]:
ae.evaluate( t_data['xt1'], preds)

In [ ]:
#intersection of unions
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

In [ ]:
print(iou_coef(t_data['xt1'], preds))

PSNR

In [ ]:
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [ ]:
print(calculate_psnr(t_data['xt0'][4].numpy(), preds[4]))

SSIM

In [ ]:
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [ ]:

print(calculate_ssim(t_data['xt0'][4].numpy(), preds[4]))

# Push on github